In [ ]:
!pip install pandas spacy
!python -m spacy download en_core_web_sm
!pip install gspread
!pip install oauth2client
!pip install fuzzywuzzy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 29.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import gspread
from google.colab import auth
import pandas as pd
import math

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
# Load the Google Sheet
#sheet_url = 'https://docs.google.com/spreadsheets/d/15VfyipYKpd53oNU2A442paDCrqXebM6-WZaqVb3uFhc/edit#gid=0'
sheet_url = 'https://docs.google.com/spreadsheets/d/15VfyipYKpd53oNU2A442paDCrqXebM6-WZaqVb3uFhc/edit?usp=sharing'
spreadsheet = gc.open_by_url(sheet_url)

worksheet = gc.open_by_url(sheet_url).sheet1

# Get all records from the sheet
records = worksheet.get_all_records()

# Convert records to DataFrame
ndf = pd.DataFrame(records)
df = ndf
ndf.head()

,Department,Job Title,Requirements
0,Accounting,Senior Tax and Accounting Specialist,"Bachelor’s Degree in Finance, Accounting, or r..."
1,Accounting,Accounting Intern,Recent graduate or currently in the last year ...
2,Accounting,Billing Manager,Proven work experience as a Billing Manager or...
3,Accounting,Accounts Payable Specialist,Proven work experience as an Accounts Payable ...
4,Accounting,Medical Biller,Proven work experience as a Medical Biller or ...


In [ ]:
import spacy
import re
from spacy.matcher import Matcher

# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

# Initialize the matcher with the spaCy model
matcher = Matcher(nlp.vocab)

# Define patterns to match specific traits and skills
pattern1 = [{"POS": "ADJ"}, {"POS": "NOUN"}]
pattern2 = [{"POS": "NOUN"}, {"POS": "NOUN"}]
matcher.add("TraitsSkills", [pattern1, pattern2])

# Function to extract traits and skills using the matcher
def extract_with_matcher(text):
    if pd.isnull(text):
        return ""
    doc = nlp(text)
    matches = matcher(doc)
    traits_skills = []
    for match_id, start, end in matches:
        span = doc[start:end]
        traits_skills.append(span.text)
    return "\n".join(set(traits_skills))


# Define a list of common personality traits
common_traits = [
    "analytical", "attention to detail", "communication", "organizational skills",
    "reliability", "team player", "problem-solving", "critical thinking", "adaptability",
    "creativity", "leadership", "empathy", "patience", "time management", "interpersonal",
    "initiative", "diligent", "rigorous", "flexible", "dependable", "hardworking"
]

# Compile the traits into a regular expression pattern
traits_pattern = re.compile(r'\b(' + '|'.join(re.escape(trait) for trait in common_traits) + r')\b', re.IGNORECASE)

# Define a function to extract traits using the compiled pattern
def new_line(text):
    clean_sents = []
    split_sents = text.split(';')  # Split by semicolons
    for s in split_sents:
        clean_sents.append(s.strip())  # Strip leading and trailing whitespace
    return ';\n'.join(clean_sents)  # Join cleaned sentences with semicolons and newlines



def extract_traits(text):

    traits = re.findall(traits_pattern, text)

    return ", ".join(trait.lower() for trait in traits)


# Function to add new lines after each trait
def add_new_lines(traits):
    return "\n".join(trait.strip() for trait in traits.split(','))



# Ensure the 'Requirements' column is treated as strings and handle NaN values
df['Requirements'] = df['Requirements'].astype(str).apply(new_line)

# Apply the matcher function to the 'Requirements' column
df['Extracted words by sentence pattern'] = df['Requirements'].apply(extract_with_matcher)

# Apply the function to the Requirements column
df['Extracted with common traits'] = df['Requirements'].apply(extract_traits)
# Apply the function to the column
#df['Formatted common Traits'] = df['Extracted with common traits'].apply(add_new_lines)
df['Formatted common Traits'] = df['Extracted with common traits']


columns = list(df.columns)
columns.remove('Extracted with common traits')
df = df[columns]





In [ ]:
# Reorder columns
columns = list(df.columns)
results_df = df[columns]

# Print results
results_df.tail()

,Department,Job Title,Requirements,Extracted words by sentence pattern,Formatted common Traits
1140,Retail,Area Manager,"Proven experience as area manager, store manag...",similar field\nExceptional communication\nbusi...,"communication, leadership, problem-solving"
1141,Retail,Retail Buyer,Proven experience as a retail buyer or relevan...,effective negotiating\nbest practices\nbusines...,"communication, organizational skills"
1142,Retail,Category Manager,Proven experience as category manager or simil...,forecasting methods\ncategory manager\nstrateg...,"analytical, communication"
1143,Retail,Merchandiser,Proven working experience in merchandising;\nH...,BS degree\ntarget audiences\nbest practices\nc...,
1144,Retail,Store Manager,Proven successful experience as a retail Store...,business orientation\nBS degree\nmanagement sk...,"organizational skills, communication, interper..."


In [ ]:
sheet_url_big5 = 'https://docs.google.com/spreadsheets/d/1ff0wQcFQM6ed0GcJf4TP52AOZvUSsU5uEeDjckJiSMw/edit#gid=0'
spreadsheet1 = gc.open_by_url(sheet_url_big5)

worksheet1 = gc.open_by_url(sheet_url_big5).sheet1

# Get all records from the sheet
records_big5 = worksheet1.get_all_records()

# Convert records to DataFrame
big_df = pd.DataFrame(records_big5)
big_df.head()

,Skill/Attribute,Openness,Conscientiousness,Extraversion,Agreeableness,Neuroticism
0,Analytical,High,High,-,-,Low
1,Attention to Detail,-,High,-,-,Low
2,Communication Skills,-,-,High,High,Low
3,Organizational Skills,Medium,High,-,-,Low
4,Reliability,-,High,-,High,-


In [ ]:
df_big5 = big_df
df_traits = results_df
df_traits.head()

,Department,Job Title,Requirements,Extracted words by sentence pattern,Formatted common Traits
0,Accounting,Senior Tax and Accounting Specialist,"Bachelor’s Degree in Finance, Accounting, or r...",accounting systems\nEnglish communication\ncom...,"communication, attention to detail, team player"
1,Accounting,Accounting Intern,Recent graduate or currently in the last year ...,last year\naccounting software\nEnglish langua...,
2,Accounting,Billing Manager,Proven work experience as a Billing Manager or...,Relevant training\nwork experience\ncollection...,communication
3,Accounting,Accounts Payable Specialist,Proven work experience as an Accounts Payable ...,Relevant training\nwork experience\naccounting...,"time management, organizational skills, commun..."
4,Accounting,Medical Biller,Proven work experience as a Medical Biller or ...,Solid understanding\nwork experience\norganiza...,"communication, problem-solving"


In [ ]:
df_traits['Formatted common Traits'] = df_traits['Formatted common Traits'].apply(lambda x: [trait.strip() for trait in x.split(',')])

In [ ]:
df_traits

,Department,Job Title,Requirements,Extracted words by sentence pattern,Formatted common Traits
0,Accounting,Senior Tax and Accounting Specialist,"Bachelor’s Degree in Finance, Accounting, or r...",accounting systems\nEnglish communication\ncom...,"[communication, attention to detail, team player]"
1,Accounting,Accounting Intern,Recent graduate or currently in the last year ...,last year\naccounting software\nEnglish langua...,[]
2,Accounting,Billing Manager,Proven work experience as a Billing Manager or...,Relevant training\nwork experience\ncollection...,[communication]
3,Accounting,Accounts Payable Specialist,Proven work experience as an Accounts Payable ...,Relevant training\nwork experience\naccounting...,"[time management, organizational skills, commu..."
4,Accounting,Medical Biller,Proven work experience as a Medical Biller or ...,Solid understanding\nwork experience\norganiza...,"[communication, problem-solving]"
...,...,...,...,...,...
1140,Retail,Area Manager,"Proven experience as area manager, store manag...",similar field\nExceptional communication\nbusi...,"[communication, leadership, problem-solving]"
1141,Retail,Retail Buyer,Proven experience as a retail buyer or relevan...,effective negotiating\nbest practices\nbusines...,"[communication, organizational skills]"
1142,Retail,Category Manager,Proven experience as category manager or simil...,forecasting methods\ncategory manager\nstrateg...,"[analytical, communication]"
1143,Retail,Merchandiser,Proven working experience in merchandising;\nH...,BS degree\ntarget audiences\nbest practices\nc...,[]


In [ ]:
# '''
# import pandas as pd
# from fuzzywuzzy import process
# import logging



# # Function to match traits with Big 5 scores
# def match_traits(traits, df_big5):
#     matched_traits = pd.DataFrame(columns=df_big5.columns)
#     for trait in traits:
#         stripped_trait = trait.strip()
#         if not stripped_trait:  # Skip if trait is empty after stripping
#              continue
#         matched_trait_score = process.extractOne(stripped_trait, df_big5['Skill/Attribute'])
#         '''process.extractOne calculate the similarity score between the input trait and the traits listed in the 'Skill/Attribute' column'''
#         if matched_trait_score:
#             matched_trait, score = matched_trait_score[:2]  # Only unpack the first two values
#             if score >= 80:  # Adjust the threshold as needed
#                 matched_traits = pd.concat([matched_traits, df_big5[df_big5['Skill/Attribute'] == matched_trait]])
#     return matched_traits

# # Function to aggregate Big 5 scores for a job
# def aggregate_personality(job_title, df_traits, df_big5):
#     job_traits = df_traits[df_traits['Job Title'] == job_title]['Formatted common Traits'].values[0] #https://chatgpt.com/share/6b3338fd-5cb1-464f-a67c-ac89132b0319
#     print(job_traits)
#     matched_traits = match_traits(job_traits, df_big5)

#     if matched_traits.empty:
#         raise ValueError(f"No matching traits found for job title '{job_title}'.")

#     # Use mode to determine the most frequent value
#     def get_mode(series):
#         mode = series.mode()
#         if not mode.empty:
#             return mode.iloc[0]
#         return '-'

#     # Aggregating the scores
#     aggregated_scores = matched_traits.iloc[:, 1:].apply(get_mode, axis=0)

#     # Override the aggregation to ensure '' doesn't overwrite valid values
#     for col in aggregated_scores.index:
#         if '-' in matched_traits[col].values:
#             valid_values = matched_traits[col][matched_traits[col] != '-']
#             if not valid_values.empty:
#                 aggregated_scores[col] = valid_values.mode().iloc[0]
#             else:
#                 aggregated_scores[col] = '-'

#     return aggregated_scores

# # Create a new DataFrame to store the results
# new_result_df = pd.DataFrame(columns=['Department','Job Title', 'Formatted common Traits', 'Openness', 'Conscientiousness', 'Extraversion', 'Agreeableness', 'Neuroticism'])

# # Iterate over each row in the DataFrame
# for index, row in df_traits.iterrows():
#     job_title = row['Job Title']
#     try:
#         personality_requirements = aggregate_personality(job_title, df_traits, df_big5)
#         result_row = {
#             'Department': row['Department'],
#             'Job Title': job_title,
#             'Formatted common Traits': row['Formatted common Traits'],
#             'Openness': personality_requirements.get('Openness', '-'),
#             'Conscientiousness': personality_requirements.get('Conscientiousness', '-'),
#             'Extraversion': personality_requirements.get('Extraversion', '-'),
#             'Agreeableness': personality_requirements.get('Agreeableness', '-'),
#             'Neuroticism': personality_requirements.get('Neuroticism', '-')
#         }
#     #print(result_row)
#         new_result_df = pd.concat([new_result_df, pd.DataFrame([result_row])], ignore_index=True)
#     except ValueError as e:
#         print()





# '''

In [ ]:
import pandas as pd
from fuzzywuzzy import process
import logging

# Suppress specific warning from fuzzywuzzy
logging.getLogger('fuzzywuzzy').setLevel(logging.ERROR)

# Function to match traits with Big 5 scores
def match_traits(traits, df_big5):
    matched_traits = pd.DataFrame(columns=df_big5.columns)
    for trait in traits:
        stripped_trait = trait.strip()
        if not stripped_trait:  # Skip if trait is empty after stripping
            continue
        matched_trait_score = process.extractOne(stripped_trait, df_big5['Skill/Attribute'])
        if matched_trait_score:
            matched_trait, score = matched_trait_score[:2]  # Only unpack the first two values
            if score >= 80:  # Adjust the threshold as needed
                matched_traits = pd.concat([matched_traits, df_big5[df_big5['Skill/Attribute'] == matched_trait]])
    return matched_traits

# Function to aggregate Big 5 scores for a job
def aggregate_personality(job_title, df_traits, df_big5):
    job_traits = df_traits[df_traits['Job Title'] == job_title]['Formatted common Traits'].values[0]
    matched_traits = match_traits(job_traits, df_big5)

    if matched_traits.empty:
        return pd.Series({'Openness': '-', 'Conscientiousness': '-', 'Extraversion': '-', 'Agreeableness': '-', 'Neuroticism': '-'})

    # Use mode to determine the most frequent value
    def get_mode(series):
        mode = series.mode()
        if not mode.empty:
            return mode.iloc[0]
        return '-'

    # Aggregating the scores
    aggregated_scores = matched_traits.iloc[:, 1:].apply(get_mode, axis=0)

    # Override the aggregation to ensure '-' doesn't overwrite valid values
    for col in aggregated_scores.index:
        if '-' in matched_traits[col].values:
            valid_values = matched_traits[col][matched_traits[col] != '-']
            if not valid_values.empty:
                aggregated_scores[col] = valid_values.mode().iloc[0]
            else:
                aggregated_scores[col] = '-'

    return aggregated_scores

# Create a new DataFrame to store the results
new_result_df = pd.DataFrame(columns=['Department','Job Title', 'Formatted common Traits', 'Openness', 'Conscientiousness', 'Extraversion', 'Agreeableness', 'Neuroticism'])

# Iterate over each row in the DataFrame
for index, row in df_traits.iterrows():
    job_title = row['Job Title']
    personality_requirements = aggregate_personality(job_title, df_traits, df_big5)
    result_row = {
        'Department': row['Department'],
        'Job Title': job_title,
        'Formatted common Traits': row['Formatted common Traits'],
        'Openness': personality_requirements.get('Openness', '-'),
        'Conscientiousness': personality_requirements.get('Conscientiousness', '-'),
        'Extraversion': personality_requirements.get('Extraversion', '-'),
        'Agreeableness': personality_requirements.get('Agreeableness', '-'),
        'Neuroticism': personality_requirements.get('Neuroticism', '-')
    }
    new_result_df = pd.concat([new_result_df, pd.DataFrame([result_row])], ignore_index=True)

# Output the resulting DataFrame
new_result_df


,Department,Job Title,Formatted common Traits,Openness,Conscientiousness,Extraversion,Agreeableness,Neuroticism
0,Accounting,Senior Tax and Accounting Specialist,"[communication, attention to detail, team player]",-,High,High,High,Low
1,Accounting,Accounting Intern,[],-,-,-,-,-
2,Accounting,Billing Manager,[communication],-,-,High,High,Low
3,Accounting,Accounts Payable Specialist,"[time management, organizational skills, commu...",Medium,High,High,High,Low
4,Accounting,Medical Biller,"[communication, problem-solving]",High,High,High,High,Low
...,...,...,...,...,...,...,...,...
1140,Retail,Area Manager,"[communication, leadership, problem-solving]",High,High,High,High,Low
1141,Retail,Retail Buyer,"[communication, organizational skills]",Medium,High,High,High,Low
1142,Retail,Category Manager,"[analytical, communication]",High,High,High,High,Low
1143,Retail,Merchandiser,[],-,-,-,-,-


In [ ]:
# Save the result to a new CSV file
'''
from google.colab import files
new_result_df.to_csv('aggregated_personality_traits.csv', index=False)
files.download('aggregated_personality_traits.csv')
'''

"\nfrom google.colab import files\nnew_result_df.to_csv('aggregated_personality_traits.csv', index=False)\nfiles.download('aggregated_personality_traits.csv')\n"

In [ ]:
from gspread_dataframe import set_with_dataframe
# Access the second sheet
worksheet2 = spreadsheet.get_worksheet(1)  # Access the second sheet (indexing starts from 0)

# Clear the sheet first if you want to overwrite the content
worksheet2.clear()

# Add the DataFrame to the sheet
set_with_dataframe(worksheet2, new_result_df)